In [1]:
import cv2
import mediapipe as mp

In [2]:
mp_hand = mp.solutions.hands#Landmarks
mp_draw = mp.solutions.drawing_utils#drawing landmarks

In [3]:
data = mp_hand.Hands(1)

In [9]:
import cv2
import mediapipe as mp
import pyautogui

# 1. Setup
screen_w, screen_h = pyautogui.size()
video = cv2.VideoCapture(0)
mp_hand = mp.solutions.hands
data = mp_hand.Hands(max_num_hands=1, min_detection_confidence=0.7)

# 2. Smoothing Variables (Storing the "Last" position)
p_t_x, p_t_y = 0, 0 # Previous Thumb X, Y
p_i_x, p_i_y = 0, 0 # Previous Index X, Y
smooth_factor = 0.2  # Lower is smoother, Higher is faster (0.1 to 0.3 is sweet spot)

while True:
    success, img = video.read()
    if not success: break
    img = cv2.flip(img, 1)
    h, w, c = img.shape
    result = data.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    if result.multi_hand_landmarks:
        for hand_lms in result.multi_hand_landmarks:
            # Get RAW normalized coordinates
            thumb = hand_lms.landmark[4]
            index = hand_lms.landmark[8]

            # Convert to Screen Pixels
            raw_t_x, raw_t_y = thumb.x * screen_w, thumb.y * screen_h
            raw_i_x, raw_i_y = index.x * screen_w, index.y * screen_h

            # --- THE SMOOTHING MATH ---
            # New Position = (Old Position) + (Difference * Smooth Factor)
            curr_t_x = p_t_x + (raw_t_x - p_t_x) * smooth_factor
            curr_t_y = p_t_y + (raw_t_y - p_t_y) * smooth_factor
            curr_i_x = p_i_x + (raw_i_x - p_i_x) * smooth_factor
            curr_i_y = p_i_y + (raw_i_y - p_i_y) * smooth_factor

            # Update Previous for next frame
            p_t_x, p_t_y, p_i_x, p_i_y = curr_t_x, curr_t_y, curr_i_x, curr_i_y

            # --- VISUALS (On Webcam) ---
            # Scale back to webcam size for drawing preview
            cam_t_x, cam_t_y = int((curr_t_x / screen_w) * w), int((curr_t_y / screen_h) * h)
            cam_i_x, cam_i_y = int((curr_i_x / screen_w) * w), int((curr_i_y / screen_h) * h)
            
            cv2.rectangle(img, (cam_t_x, cam_t_y), (cam_i_x, cam_i_y), (0, 255, 0), 2)
            cv2.circle(img, (cam_t_x, cam_t_y), 5, (255, 0, 0), -1)
            cv2.circle(img, (cam_i_x, cam_i_y), 5, (255, 0, 0), -1)

    cv2.imshow("Smoothed Box Preview", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

video.release()
cv2.destroyAllWindows()